In [ ]:
# default_exp assemble

# assemble

> High-level functions to tidy up already parsed data

In [ ]:
# export
import pathlib

import pandas as pd

# import dlsproc.xml
# import dlsproc.structure
import dlsproc.bundle
import dlsproc.hier

Directory where the zip files are stored

In [ ]:
directory = pathlib.Path.cwd() / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/dlsproc/samples')

### Zip file

A (sample) file in that directory

In [ ]:
input_file = directory / 'PlataformasAgregadasSinMenores_202201_05-06.zip'
assert input_file.exists()
input_file

PosixPath('/home/manu/dlsproc/samples/PlataformasAgregadasSinMenores_202201_05-06.zip')

In [ ]:
df = dlsproc.hier.flat_df_to_multiindexed_df(dlsproc.bundle.read_zip(input_file, concatenate=True))
df.head(2)

id  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      https://contrataciondelestado.es/sindicacion/P...   
                                                   1      https://contrataciondelestado.es/sindicacion/P...   

                                                                                                    summary  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      Id licitación: 2021039438; Órgano de Contratac...   
                                                   1      Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                      title  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          entry                                                      
PlataformasAgregadasSinMenores_20220105_030012.... 0      L'objecte és la contractació del servei de bug...   
                                                   1      Servei comunicacions postals de l'Ajuntament d...   

                                                                                  updated  \
                                                                                            
                                                                                            
                                                                                            
                                                                                            
                                                                                            
file name                                          entry                                    
PlataformasAgregadasSinMenores_20220105_030012.... 0     2022-01-04 12:12:09.464000+00:00   
                                                   1     2022-01-04 12:12:09.400000+00:00   

                                                         ContractFolderStatus  \
                                                             ContractFolderID   
                                                                                
                        

The last few columns

In [ ]:
df.columns[-2:]

MultiIndex([('ContractFolderStatus', 'TenderingProcess', ...),
            ('ContractFolderStatus', 'TenderingProcess', ...)],
           )

Deleted contracts are extracted

In [ ]:
deleted_df = dlsproc.bundle.read_deleted_zip(input_file)
deleted_df.head(2)

file name                                            id                                                                                 
PlataformasAgregadasSinMenores_20220105_030012.atom  https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/6724977   2022-01-04 01:12:01.376000+01:00
                                                     https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/1968315   2022-01-04 00:11:57.567000+01:00
Name: deleted_on, dtype: datetime64[ns, pytz.FixedOffset(60)]

## Handling deleted entries

In [ ]:
# export
def merge_deleted(data_df: pd.DataFrame, deleted_series: pd.Series) -> pd.DataFrame:
    
    # duplicates are dropped; in order to do so it is convenient to turn the `pd.Series` into a `pd.DataFrame` by calling
    # `reset_index`, which turns the *multiindex* into columns
    deduplicated_deleted_df = deleted_series.reset_index().drop_duplicates('id').set_index(['file name', 'id'])
    
    # in order to merge this new `pd.DataFrame` with `data_df` we need a *multiindex* for the former with the same number of levels as in the latter
    deduplicated_deleted_df.columns = col_multiindex = pd.MultiIndex.from_tuples([dlsproc.hier.pad_col_levels(data_df, ['deleted_on'])])
    
    # the `data_df` is (*left*-)joined with the new one yielding deleted entries; the result is a *stateful* `pd.DataFrame` in the sense that,
    # for every entry, we know its state: deleted or not; notice that `data_df`'s index is reset for easying the merge (assuming every contract
    # shows only once in `data_df`, *file name* and *id* should still provide a unique index...though it probably doesn't matter anyway)
    res = data_df.reset_index().set_index(['file name', 'id']).merge(deduplicated_deleted_df, how='left', on=['file name', 'id'])
    
    return res

In [ ]:
merged_df = merge_deleted(df, deleted_df)
merged_df.head(2)

entry  \
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
                                                                                                              
file name                                          id                                                         
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...     0   
                                                   https://contrataciondelestado.es/sindicacion/Pl...     1   

                                                                                                                                                 summary  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2021039438; Órgano de Contratac...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Id licitación: 2021_2568; Órgano de Contrataci...   

                                                                                                                                                   title  \
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
                                                                                                                                                           
file name                                          id                                                                                                      
PlataformasAgregadasSinMenores_20220105_030012.... https://contrataciondelestado.es/sindicacion/Pl...  L'objecte és la contractació del servei de bug...   
                                                   https://contrataciondelestado.es/sindicacion/Pl...  Servei comunicacions postals de l'Ajuntament d...   

                                                                                                                               updated  \
                                                                                                                                         
                

The new *merged* `pd.DataFrame` has the same number of rows as the original one

In [ ]:
assert len(df) == len(merged_df)

How many entries have been deleted?

In [ ]:
merged_df['deleted_on'].notna().sum()

0

Some examples

In [ ]:
# merged_df[merged_df['deleted_on'].notna()].head()

In [ ]:
# hide
import nbdev.export
nbdev.export.notebook2script('60_assemble.ipynb')

Converted 60_assemble.ipynb.
